In [1]:
import nltk
import sys
reload(sys)
sys.setdefaultencoding("utf-8")

In [29]:
def extract_entity_names(t):
    entity_names = []
    
    if hasattr(t, 'node') and t.node:
        if t.node == 'NE':
            entity_names.append(' '.join([child[0] for child in t]))
        else:
            for child in t:
                entity_names.extend(extract_entity_names(child))
                
    return entity_names


with open('text/shipman5.txt', mode='r') as f:
    sample = f.read().decode('utf8')  
    sentences = nltk.sent_tokenize(sample)
    tokenized_sentences = [nltk.word_tokenize(sentence) for sentence in sentences]
    tagged_sentences = [nltk.pos_tag(sentence) for sentence in tokenized_sentences]
    chunked_sentences = nltk.ne_chunk_sents(tagged_sentences, binary=True)

entity_names = []
for tree in chunked_sentences:
    # Print results per sentence
    # print extract_entity_names(tree)    
    entity_names.extend(extract_entity_names(tree))

# Print all entity names
#print entity_names

# Print unique entity names
print(set(entity_names))

In [20]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [7]:
os.listdir('./text')

['shipman4.out.txt',
 'shipman1.out.txt',
 'shipman3.out.txt',
 'shipman2.out.txt',
 'shipman5.out.txt',
 'shipman6.out.txt']

In [1]:
# textblob usage example
import os
from textblob import TextBlob

blobs = []
for textfile in os.listdir('./tagged_text'):    
    with open('./tagged_text/' + textfile) as f:
        text = f.read().decode('utf8')           
        t = TextBlob(text)
        blobs.append(t)        

In [49]:
import re
r = re.compile(r"<PERSON>.*?<\/PERSON>")

correlations = []
# get named entities correlations
with open('./tagged_text/' + textfile) as f:
    for line in iter(f):        
        correlation = r.findall(line)        
        #print(len(correlation))        
        if len(correlation) > 1:                                      
            correlations.append(correlation)            

In [68]:
def strip_persontag(text):
    import re
    r = re.compile(r"<PERSON>(.*)<\/PERSON>")
    m = r.match(text)
    return m.group(1)    

In [50]:
# Create a set of tuple
unique_correlations = set()
nodes = set()

In [51]:
for correlation in correlations:
    for i in range(len(correlation)-1):  
        nodes.add(correlation[i])
        nodes.add(correlation[i+1])
        pair = (correlation[i],correlation[i+1])
        unique_correlations.add(pair)

In [52]:
len(unique_correlations)

690

In [53]:
len(nodes)

558

In [71]:
import csv
cw = csv.writer(open("edge.csv",'w'))
for pair in unique_correlations:
    cw.writerow([pair[0],pair[1],"Edge from " + strip_persontag(pair[0]) + " from " + strip_persontag(pair[1])])

In [70]:
cw = csv.writer(open("nodes.csv", 'w'))
for node in nodes:
    cw.writerow([node,strip_persontag(node)])

In [75]:
# lda

In [111]:
import os
import json
import pandas as pd
import re

In [112]:
from nltk.stem import WordNetLemmatizer
def tokenize(text):
    result_words = []
    wordnet_lemmatizer = WordNetLemmatizer()
    words = re.sub(r'[\W_\d]', ' ', text).lower().split()
    for word in words:
        result_words.append(wordnet_lemmatizer.lemmatize(word))
    return result_words

In [128]:
from sklearn.feature_extraction.text import TfidfVectorizer

documents_text = []
for textname in os.listdir('./text/'):
    with open('./text/' + textname) as f:
        document_text = f.read().decode('utf8')
        documents_text.append(document_text)        
        
#vectorizer = TfidfVectorizer(stop_words='english', tokenizer = tokenize)
vectorizer = TfidfVectorizer(stop_words='english')
termsDocumentsMatrix = vectorizer.fit_transform(documents_text)

In [129]:
print(len(vectorizer.get_feature_names()))
terms = vectorizer.get_feature_names()

16022


In [130]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=5, n_iter=7, random_state=42)
svdMatrix = svd.fit_transform(termsDocumentsMatrix)

In [131]:
svd.components_.shape

(5, 16022)

In [132]:
# find top terms for each concept
import numpy as np

for i in range(0, svd.components_.shape[0]):    
    sortedTerms = np.argsort(svd.components_[i])
    print "Concept:"
    for k in range(10):
        print terms[sortedTerms[k]]
    print "\n\n\n"

Concept:
zeebrugge
leniency
oversaw
aides
lesion
epidemic
envisaging
environments
proponent
uptake




Concept:
death
di
deaths
shipman
coroner
dr
smith
mrs
reynolds
cause




Concept:
drugs
shipman
controlled
death
patient
wards
pgi
drug
ward
home




Concept:
death
doctor
gmc
medical
wards
pgi
case
shipman
cases
ward




Concept:
shipman
di
wards
pgi
patients
ward
smith
gmc
patient
mrs




